In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE CUSTOMERS_RAW
AS SELECT *,current_timestamp() as load_time
FROM cloud_files("/Volumes/dev/demo_db/dlt_data/customers/","csv",map("cloudFiles.inferColumnTypes","true"))

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE INVOICES_RAW
AS SELECT *,current_timestamp() as load_time
FROM cloud_files("/Volumes/dev/demo_db/dlt_data/invoices/","csv",map("cloudFiles.inferColumnTypes","true"))

## Create your Silver layer Tables reading incremental data from bronze layer

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE INVOICES_CLEANED(
  CONSTRAINT valid_invoice_and_qty EXPECT (invoice_no IS NOT NULL and quantity > 0) ON VIOLATION DROP ROW)
  PARTITIONED BY(invoices_year,country)
AS 
SELECT InvoiceNo as invoice_no, StockCode as stock_code, Description as description,
        Quantity as quantity, to_date(InvoiceDate,"d-M-y H.m") as invoice_date,
        UnitPrice as unit_price, CustomerID as customer_id, Country as country,
        year(to_date(invoice_date,"d-M-y H.m")) as invoices_year, month(to_date(invoice_date,"d-M-y H.m")) as invoices_month,
        load_time
FROM STREAM(live.INVOICES_RAW)

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE CUSTOMER_CLEANED(
  CONSTRAINT valid_customer EXPECT (customer_id IS NOT NULL) ON VIOLATION DROP ROW
) AS SELECT CustomerID as customer_id, CustomerName as customer_name, load_time 
FROM STREAM(live.CUSTOMERS_RAW)

## Implementing SCD Type 2 for the Silver Layer

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE CUSTOMERS;

APPLY CHANGES INTO live.CUSTOMERS
FROM STREAM(live.CUSTOMER_CLEANED)
KEYS(customer_id)
SEQUENCE BY load_time
STORED AS SCD TYPE 2;

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE INVOICES PARTITIONED BY(invoices_year,country);

APPLY CHANGES INTO live.INVOICES
FROM STREAM(live.INVOICES_CLEANED)
KEYS(invoice_no,stock_code,invoice_date)
SEQUENCE BY load_time;

## Materialize the Gold Layer

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE DAILY_SALES_UK_2022
AS
SELECT country,invoice_year,invoice_month,invoice_date,
      round(sum(quantity*unit_price),2) as total_sales
FROM LIVE.INVOICES
WHERE invoice_year = 2022 AND country = 'United Kingdom'
GROUP BY country,invoice_year,invoice_month,invoice_date